In [0]:
#@title Connec to Google Drive & Load Libraries

!pip -q install geoip2
!apt-get -o Dpkg::Options::="--force-confmiss" install --reinstall netbase
!pip -q install folium==0.2.1
!pip -q install pandas-profiling[notebook,html]

import sys, warnings
from google.colab import drive, files

print("\nGoogle Drive File Stream Authentication\n")
drive.mount('/content/drive',force_remount=True)
sys.path.append('/content/drive/My Drive')
sys.path.append('/content/drive/My Drive/Colab Notebooks/Notebooks')
warnings.filterwarnings("ignore")

from lib.DatasetPreprocessingController import *

rcParams['figure.figsize'] = 15, 8
sns.set(style="whitegrid")

In [0]:
#@title #Settings { run: "auto" }

#@markdown Name of dataset header file
datasetHeader = 'NUSW-NB15_features' #@param {type:"string"}
#@markdown ---
#@markdown Name of the dataset to load
datasetToLoad = 'UNSW-NB15_4' #@param {type:"string"}
#@markdown ---
#@markdown Name of the dataset to be saved
datasetToSave = '' #@param {type:"string"}
#@markdown ---
#@markdown Set the name of the notebook
notebookName = '' #@param {type:"string"}
#@markdown ---
#@markdown \* Type file names without file extension

#@markdown

#@markdown  ##Dataset preprocessing settings
ipLocationMapping = 'numeric' #@param ["numeric", "nominal"]
portTypeMapping = 'numeric' #@param ["numeric", "nominal"]
protocolMapping = 'numeric' #@param ["numeric", "nominal"]
keepShortRangeOfProtocols = False #@param ["False", "True"] {type:"raw"}
stateMapping = 'numeric' #@param ["numeric", "nominal"]
serviceMapping = 'numeric' #@param ["numeric", "nominal"]
attackCategoryMapping = 'numeric' #@param ["numeric", "nominal"]
OHE = False #@param {type:"boolean"}
resampling = False #@param {type:"boolean"}

In [0]:
#@title ###Loading dataset
dataset = loadData(datasetHeader, datasetToLoad)

In [0]:
#@title ###Dataset header
dataset.head()

In [0]:
#@title ###Describe dataset
dataset.describe()

In [0]:
#@title ###Normal vs Attack class distibution
count_classes = pd.value_counts(dataset.Label)
count_classes.plot(kind="bar")
plt.title("Normal vs attack class distibution")
plt.xticks(range(2), ["Normal","Attack"])
plt.xlabel("Class")
plt.ylabel("Frequency")

attack_traffic=dataset[dataset.Label.apply(lambda x: x==1)]
normal_traffic=dataset[dataset.Label.apply(lambda x: x==0)]
print("Normal traffic: {}\nAttack traffic: {}".format(normal_traffic.shape, attack_traffic.shape))

In [0]:
#@title ###Missing values
nan_rows = dataset.columns[dataset.isna().any()].tolist()
print("NaN columns {}\n".format(nan_rows))

for r in nan_rows:
  nan_row = dataset[dataset[r].isnull()]
  nan_norm = nan_row[nan_row.Label == 0]
  nan_att = nan_row[nan_row.Label == 1]
  print("\'{}\'\nNormal: {}\nAttack: {}\n".format(r,nan_norm.shape[0],nan_att.shape[0]))

dataset.attack_cat = dataset.attack_cat.fillna('Normal')

In [0]:
#@title ###Protocol count
dataset.proto.value_counts()

In [0]:
#@title ###State count
dataset.state.value_counts()

In [0]:
#@title ###Renamig values for state
print("Renaming rows in \'state\' column with value \'no\' to \'NMU\' (Not Much Used)\n")
dataset.state = dataset.state.replace(to_replace ="no", value ="NMU")

#print("Removing rows in \'state\' column with value \'no\'\n")
#dataset = dataset[dataset.state.apply(lambda x: x != 'no')]

In [0]:
#@title ###Service count
dataset.service.value_counts()

In [0]:
#@title ###Renaming values for service
print("Renaming rows in \'service\' column with value \'-\' to \'nmu\' (Not Much Used)\n")
dataset.service = dataset.service.replace(to_replace ="-", value ="nmu");

In [0]:
#@title ###List of not defined source ports
print("List of not defined source port")
print(dataset[dataset.sport.apply(lambda x: x == '-')])

In [0]:
#@title ###Removing not defined source port entries
print("Removing not defined source port entries")
dataset = dataset[dataset.sport.apply(lambda x: x != '-')]

In [0]:
#@title ###List of not defined destination ports
print("List of not defined destination port")
dataset[dataset.dsport.apply(lambda x: x == '-')]

In [0]:
#@title ###Removing not defined source port entries
print("Removing not defined source port entries")
dataset = dataset[dataset.dsport.apply(lambda x: x != '-')]

In [0]:
#@title ###Converting hexadecimal source/destination port number to decimal
try:
  print("Converting hexadecimal source port number to decimal")
  dataset.sport = dataset.sport.apply(lambda x: int(x,16) if re.search('0x.*', x) else x)
except TypeError:
  print("No hexadecimal source port number")

try:
  print("Converting hexadecimal destination port number to decimal")
  dataset.dsport = dataset.dsport.apply(lambda x: int(x,16) if re.search('0x.*', x) else x)
except TypeError:
  print("No hexadecimal destination port number")


In [0]:
#@title ###Removing entries with port number larger than 65 535
print("Removing entries with port number larger than 65535")
dataset = dataset[dataset.sport.apply(lambda x: int(x) <= 65535)]
dataset = dataset[dataset.dsport.apply(lambda x: int(x) <= 65535)]

In [0]:
#@title ###Duration analysis and processing
dur_treshold = 8000 #@param {type:"number"}
mean, std, median, treshold = makePlot(dataset.index.astype(int),dataset.dur,"Record total duration",dur_treshold)
print("\nMean: {}\nSTD: {}\nMedian: {}\nTreshold: {}\n".format(mean,std,median,treshold))

test=dataset[dataset.dur.astype(float).apply(lambda x: x>dur_treshold)]
print("Normal traffic: {}".format(test[test.Label.apply(lambda x: x==0)].shape))
print("Attack traffic: {}".format(test[test.Label.apply(lambda x: x==1)].shape))

print("Removing duration longer than %d ms" % dur_treshold)
dataset = dataset[dataset.dur.apply(lambda x: float(x) < dur_treshold)]
#q = dataset.dur.quantile(0.999995)
#test_d=dataset[dataset.dur < q]

In [0]:
#@title ###Source to destination transaction bytes analysis and processing
sbytes_treshold = 2000000 #@param {type:"number"}
mean, std, median, treshold = makePlot(dataset.index.astype(int),dataset.sbytes,"Source to destination transaction bytes ",sbytes_treshold)
print("\nMean: {}\nSTD: {}\nMedian: {}\nTreshold: {}\n".format(mean,std,median,treshold))

test=dataset[dataset.sbytes.astype(float).apply(lambda x: x>sbytes_treshold)]
print("Normal traffic: {}".format(test[test.Label.apply(lambda x: x==0)].shape))
print("Attack traffic: {}".format(test[test.Label.apply(lambda x: x==1)].shape))

In [0]:
#@title ###Destination to source transaction bytes analysis and processing
dbytes_treshold = 2000000 #@param {type:"number"}
mean, std, median, treshold = makePlot(dataset.index.astype(int),dataset.dbytes,"Destination to source transaction bytes ",dbytes_treshold)
print("\nMean: {}\nSTD: {}\nMedian: {}\nTreshold: {}\n".format(mean,std,median,treshold))

test=dataset[dataset.dbytes.astype(float).apply(lambda x: x>dbytes_treshold)]
print("Normal traffic: {}".format(test[test.Label.apply(lambda x: x==0)].shape))
print("Attack traffic: {}".format(test[test.Label.apply(lambda x: x==1)].shape))

In [0]:
#@title ###Source to destination time to live values analysis and processing
sttl_treshold = 0 #@param {type:"number"}
mean, std, median, treshold = makePlot(dataset.index.astype(int),dataset.sttl,"Source to destination time to live value ",sttl_treshold)
print("\nMean: {}\nSTD: {}\nMedian: {}\nTreshold: {}\n".format(mean,std,median,treshold))

test=dataset[dataset.sttl.astype(float).apply(lambda x: x>sttl_treshold)]
print("Normal traffic: {}".format(test[test.Label.apply(lambda x: x==0)].shape))
print("Attack traffic: {}".format(test[test.Label.apply(lambda x: x==1)].shape))

In [0]:
#@title ###Destination to source time to live values analysis and processing
dttl_treshold = 0 #@param {type:"number"}
mean, std, median, treshold = makePlot(dataset.index.astype(int),dataset.dttl,"Destination to source time to live value ",dttl_treshold)
print("\nMean: {}\nSTD: {}\nMedian: {}\nTreshold: {}\n".format(mean,std,median,treshold))

test=dataset[dataset.dttl.astype(float).apply(lambda x: x>dttl_treshold)]
print("Normal traffic: {}".format(test[test.Label.apply(lambda x: x==0)].shape))
print("Attack traffic: {}".format(test[test.Label.apply(lambda x: x==1)].shape))

In [0]:
#@title ###Source bits per second analysis and processing
sload_treshold = 3000000000 #@param {type:"number"}
mean, std, median, treshold = makePlot(dataset.index.astype(int),dataset.Sload,"Source bits per second",sload_treshold)
print("\nMean: {}\nSTD: {}\nMedian: {}\nTreshold: {}\n".format(mean,std,median,treshold))

test=dataset[dataset.Sload.astype(float).apply(lambda x: x>sload_treshold)]
print("Normal traffic: {}".format(test[test.Label.apply(lambda x: x==0)].shape))
print("Attack traffic: {}".format(test[test.Label.apply(lambda x: x==1)].shape))

In [0]:
#@title ###Destination bits per second analysis and processing
dload_treshold = 17000000 #@param {type:"number"}
mean, std, median, treshold = makePlot(dataset.index.astype(int),dataset.Dload,"Destination bits per second",dload_treshold)
print("\nMean: {}\nSTD: {}\nMedian: {}\nTreshold: {}\n".format(mean,std,median,treshold))

test=dataset[dataset.Dload.astype(float).apply(lambda x: x>dload_treshold)]
print("Normal traffic: {}".format(test[test.Label.apply(lambda x: x==0)].shape))
print("Attack traffic: {}".format(test[test.Label.apply(lambda x: x==1)].shape))

In [0]:
#@title ###Source to destination packet count analysis and processing
spkts_treshold = 1500 #@param {type:"number"}
mean, std, median, treshold = makePlot(dataset.index.astype(int),dataset.Spkts,"Source to destination packet count",spkts_treshold)
print("\nMean: {}\nSTD: {}\nMedian: {}\nTreshold: {}\n".format(mean,std,median,treshold))

test=dataset[dataset.Spkts.astype(float).apply(lambda x: x>spkts_treshold)]
print("Normal traffic: {}".format(test[test.Label.apply(lambda x: x==0)].shape))
print("Attack traffic: {}".format(test[test.Label.apply(lambda x: x==1)].shape))

In [0]:
#@title ###Destination to source packet count analysis and processing
dpkts_treshold = 1500 #@param {type:"number"}
mean, std, median, treshold = makePlot(dataset.index.astype(int),dataset.Dpkts,"Destination to source packet count",dpkts_treshold)
print("\nMean: {}\nSTD: {}\nMedian: {}\nTreshold: {}\n".format(mean,std,median,treshold))

test=dataset[dataset.Dpkts.astype(float).apply(lambda x: x>dpkts_treshold)]
print("Normal traffic: {}".format(test[test.Label.apply(lambda x: x==0)].shape))
print("Attack traffic: {}".format(test[test.Label.apply(lambda x: x==1)].shape))

In [0]:
#@title ###Mean of the how packet size transmitted by the src
smeansz_treshold = 0 #@param {type:"number"}
mean, std, median, treshold = makePlot(dataset.index.astype(int),dataset.smeansz,"Mean of the how packet size transmitted by the src",smeansz_treshold)
print("\nMean: {}\nSTD: {}\nMedian: {}\nTreshold: {}\n".format(mean,std,median,treshold))

In [0]:
#@title ###Mean of the how packet size transmitted by the dst
dmeansz_treshold = 0 #@param {type:"number"}
mean, std, median, treshold = makePlot(dataset.index.astype(int),dataset.dmeansz,"Mean of the how packet size transmitted by the dst",dmeansz_treshold)
print("\nMean: {}\nSTD: {}\nMedian: {}\nTreshold: {}\n".format(mean,std,median,treshold))

In [0]:
#@title ###Pipelined depth into the connection of http request/response transaction analysis and processing
trans_depth_treshold = 3.9 #@param {type:"number"}
mean, std, median, treshold = makePlot(dataset.index.astype(int),dataset.trans_depth,"Represents the pipelined depth into the connection of http request/response transaction",trans_depth_treshold)
print("\nMean: {}\nSTD: {}\nMedian: {}\nTreshold: {}\n".format(mean,std,median,treshold))

test=dataset[dataset.trans_depth.astype(float).apply(lambda x: x>trans_depth_treshold)]
print("Normal traffic: {}".format(test[test.Label.apply(lambda x: x==0)].shape))
print("Attack traffic: {}".format(test[test.Label.apply(lambda x: x==1)].shape))

In [0]:
#@title ###Source interpacket arrival time (mSec) analysis and processing
sintpkt_treshold = 50000 #@param {type:"number"}
mean, std, median, treshold = makePlot(dataset.index.astype(int),dataset.Sintpkt,"Source interpacket arrival time (mSec)",sintpkt_treshold)
print("\nMean: {}\nSTD: {}\nMedian: {}\nTreshold: {}\n".format(mean,std,median,treshold))

test=dataset[dataset.Sintpkt.astype(float).apply(lambda x: x>sintpkt_treshold)]
print("Normal traffic: {}".format(test[test.Label.apply(lambda x: x==0)].shape))
print("Attack traffic: {}".format(test[test.Label.apply(lambda x: x==1)].shape))

In [0]:
#@title ###Source interpacket arrival time (mSec) analysis and processing
dintpkt_treshold = 50000 #@param {type:"number"}
mean, std, median, treshold = makePlot(dataset.index.astype(int),dataset.Dintpkt,"Destination interpacket arrival time (mSec)",dintpkt_treshold)
print("\nMean: {}\nSTD: {}\nMedian: {}\nTreshold: {}\n".format(mean,std,median,treshold))

test=dataset[dataset.Dintpkt.astype(float).apply(lambda x: x>dintpkt_treshold)]
print("Normal traffic: {}".format(test[test.Label.apply(lambda x: x==0)].shape))
print("Attack traffic: {}".format(test[test.Label.apply(lambda x: x==1)].shape))

In [0]:
#@title ###Correlation of features - data mapping
dataset.rename(columns={"ct_src_ ltm": "ct_src_ltm"}, inplace=True);
makeMultiPlot(dataset.index.astype(int),dataset.ct_src_ltm,dataset.index.astype(int),dataset.ct_dst_ltm,'ct_src_ltm','ct_dst_ltm',"No. of connections of the same source/destination address in 100 connections according to the last time.")

In [0]:
#@title ###Correlation of features
corr, _ = pearsonr(dataset.sloss, dataset.sbytes)
print('Pearsons correlation [sloss, sbytes]: %.3f' % corr)

corr, _ = pearsonr(dataset.dloss, dataset.dbytes)
print('Pearsons correlation [dloss, dbytes]: %.3f' % corr)

In [0]:
#@title ###Correlation matrix of selected features
features=list(['ct_srv_src','ct_srv_dst','ct_dst_ltm','ct_src_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm'])
corrmat = dataset[features].corr()
top_corr_features = corrmat.index
pl.figure(figsize=(20,20))
g=sns.heatmap(dataset[top_corr_features].corr(method='pearson', min_periods=1),annot=True,cmap="RdYlGn")

In [0]:
#@title ###Source and destination port type processing
if ('sporttype' or 'dsporttype') not in dataset.columns:
  dataset.insert(2, "sporttype", None)
  dataset.insert(5, "dsporttype", None)

print("Ports range\n"+
      "Well-know: 0 - 1023 (1)\n"+
      "Registered: 1024-49151 (2)\n"+
      "Private: 49152-65535 (3)\n")

if portTypeMapping == 'numeric':
  dataset.sporttype = dataset.sport.astype(int).apply(lambda x: portTypeNumeric(x))
  dataset.dsporttype = dataset.dsport.astype(int).apply(lambda x: portTypeNumeric(x))
else:
  dataset.sporttype = dataset.sport.astype(int).apply(lambda x: portTypeNominal(x))
  dataset.dsporttype = dataset.dsport.astype(int).apply(lambda x: portTypeNominal(x))

dataset.head()

attack_traffic=dataset[dataset.Label.apply(lambda x: x==1)]
normal_traffic=dataset[dataset.Label.apply(lambda x: x==0)]

print("Source port type\n\nNormal flow\n")
print(normal_traffic.sporttype.value_counts())
print("\nAttack flow\n")
print(attack_traffic.sporttype.value_counts())
print("\n")
print("Destination port type\n\nNormal flow\n")
print(normal_traffic.dsporttype.value_counts())
print("\nAttack flow\n")
print(attack_traffic.dsporttype.value_counts())

In [0]:
#@title ###Protocol processing

#path = dataset_path+"protocol-numbers.csv";
#protocol_numbers = pd.read_csv(path, delimiter=',', encoding='utf-8', low_memory=False, skipinitialspace=True, skip_blank_lines=True, verbose=False);
#protocol_numbers=protocol_numbers[protocol_numbers.Keyword.notnull()];
#protocol_numbers.reindex();

#for p in protocols:
#  n=-1
#  for r in protocol_numbers.itertuples():
#    if r.Keyword.lower() == p:
#      n = int(r.Decimal);
#      break
#  protocols_json[p] = n;
#protocols_mapping = {"udp": 17, "arp": 143, "tcp": 6, "ospf": 89, "icmp": 1, "igmp": 2, "sctp": 132, "udt": 144, "sep": 145, "sun-nd": 77, "swipe": 53, "mobile": 55, "pim": 103, "rtp": 146, "ipnip": 4, "ip": 147, "ggp": 3, "st2": 5, "egp": 8, "cbt": 7, "emcon": 14, "nvp": 11, "igp": 9, "xnet": 15, "argus": 13, "bbn-rcc": 10, "chaos": 16, "pup": 12, "hmp": 20, "mux": 18, "dcn": 19, "prm": 21, "trunk-1": 23, "xns-idp": 22, "trunk-2": 24, "leaf-1": 25, "leaf-2": 26, "irtp": 28, "rdp": 27, "iso-tp4": 29, "netblt": 30, "mfe-nsp": 31, "merit-inp": 32, "3pc": 34, "xtp": 36, "idpr": 35, "tp++": 39, "ddp": 37, "idpr-cmtp": 38, "ipv6": 41, "il": 40, "idrp": 45, "ipv6-frag": 44, "sdrp": 42, "ipv6-route": 43, "gre": 47, "rsvp": 46, "mhrp": 148, "bna": 49, "i-nlsp": 52, "narp": 54, "ipv6-no": 59, "tlsp": 56, "skip": 57, "ipv6-opts": 60, "any": 61, "cftp": 62, "sat-expak": 64, "kryptolan": 65, "rvd": 66, "ippc": 67, "sat-mon": 69, "ipcv": 71, "visa": 70, "cpnx": 72, "cphb": 73, "wsn": 74, "pvp": 75, "br-sat-mon": 76, "wb-mon": 78, "wb-expak": 79, "iso-ip": 80, "secure-vmtp": 82, "vmtp": 81, "vines": 83, "ttp": 84, "nsfnet-igp": 85, "dgp": 86, "tcf": 87, "eigrp": 88, "sprite-rpc": 90, "larp": 91, "mtp": 92, "ax.25": 93, "ipip": 94, "micp": 95, "aes-sp3-d": 149, "encap": 98, "etherip": 97, "pri-enc": 150, "gmtp": 100, "pnni": 102, "ifmp": 101, "aris": 104, "qnx": 106, "a/n": 107, "scps": 105, "snp": 109, "ipcomp": 108, "compaq-peer": 110, "ipx-n-ip": 111, "vrrp": 112, "zero": 151, "pgm": 113, "iatp": 117, "ddx": 116, "l2tp": 115, "srp": 119, "stp": 118, "smp": 121, "uti": 120, "sm": 122, "ptp": 123, "fire": 125, "crtp": 126, "isis": 152, "crudp": 127, "sccopmce": 153, "sps": 130, "pipe": 131, "iplt": 129, "unas": 154, "fc": 133, "ib": 155};

if keepShortRangeOfProtocols:
  regex = r'^(?!(\btcp\b|\budp\b|\barp\b|\bospf\b|\bicmp\b|\bgre\b|\bsctp\b)).*$'
  dataset.proto = dataset.proto.replace(to_replace=regex, value='other', regex=True)

if protocolMapping == 'numeric':
  protocols = dataset.proto.value_counts().index.tolist()
  protocols_json = {};

  i=1
  for p in protocols:
    protocols_json[p]=i
    i+=1
  
  dataset = dataset.replace({'proto': protocols_json})
  print("Protocol mapping\n{}".format(protocols_json))

In [0]:
#@title ###Attack categories processing
def isString(item):
  if type(item) is str:
    return item.strip()
  else: return item

if attackCategoryMapping == 'numeric':
  attackCategories = dataset.attack_cat.value_counts().index.tolist()
  attackCategories_json = {};

  i=0
  for p in attackCategories:
    p = p.strip()
    attackCategories_json[p]=i
    i+=1

  dataset.attack_cat = dataset.attack_cat.apply(lambda x: isString(x))
  dataset = dataset.replace({'attack_cat': attackCategories_json})
  print("Attack category mapping\n{}".format(attackCategories_json))

  attacks=dataset.loc[dataset.attack_cat.apply(lambda x: x > 0)]
else:
  attacks=dataset.loc[dataset.attack_cat.apply(lambda x: x is not 'Normal')]


print("Unique values and count in \'attack_cat\' column\n")
dataset.attack_cat.value_counts()
# for table view -> dataset.groupby(['attack_cat']).size().reset_index(name='counts').sort_values(by='counts', ascending=False)

plt.figure()
colors = ['red','orange','lightgreen','darkgreen','lightblue','blue','yellow','pink','purple']
attacks.attack_cat.value_counts().plot(kind='bar', title='Attack Distribution', color=colors)
category_labels=['Normal','Generic','Exploits','Fuzzers','Reconnaissance','DoS','Backdoors','Analysis','Shellcode','Worms']
plt.xticks(range(len(category_labels)), category_labels)
plt.xlabel("Category")
plt.ylabel("Frequency")
plt.show()

In [0]:
#@title ###State processing
if stateMapping == 'numeric':
  states = dataset.state.value_counts().index.tolist()
  states_json = {};

  i=1
  for s in states:
    states_json[s]=i
    i+=1

  dataset = dataset.replace({'state': states_json})
  print("State mapping\n{}".format(states_json))

In [0]:
#@title ###Service processing
if stateMapping == 'numeric':
  services = dataset.service.value_counts().index.tolist()
  services_json = {};

  i=1
  for s in services:
    services_json[s]=i
    i+=1

  dataset = dataset.replace({'service': services_json})
  print("State mapping\n{}".format(services_json))

service_col=dataset.service;
dataset=dataset.drop(columns='service');
dataset.insert(6, "service",service_col);

In [0]:
#@title ###Source and destination IP address processing
def union(a,b):
  for c in b:
    if c not in a:
      a.append(c)
  return a

if ('srciploc' or 'dstiploc') not in dataset.columns:
  dataset.insert(1, "srciploc", None)
  dataset.insert(5, "dstiploc", None)

geo_path = resources_path+'GeoLite2-Country.mmdb';
reader=geoip2.database.Reader(geo_path);

dataset.srciploc=dataset.srcip.apply(lambda x: getIPLocation(reader, x));
dataset.dstiploc=dataset.dstip.apply(lambda x: getIPLocation(reader, x));

if ipLocationMapping == 'numeric':
  srcip = dataset.srciploc.value_counts().index.tolist()
  destip = dataset.dstiploc.value_counts().index.tolist()

  merged_iploc = union(srcip,destip)
  merged_iploc_json = {};

  i=1
  for ip in merged_iploc:
    merged_iploc_json[ip]=i
    i+=1

  dataset = dataset.replace({'srciploc': merged_iploc_json})
  dataset = dataset.replace({'dstiploc': merged_iploc_json})
  print("IP location mapping\n{}".format(merged_iploc_json))

dataset.srcip = dataset.srciploc
dataset.dstip = dataset.dstiploc
dataset = dataset.drop(columns=['srciploc','dstiploc'])

In [0]:
#@title ###Backing up dataset
dataset.reindex()
dataset.index = range(len(dataset))
dataset.rename(columns={"Label": "label"}, inplace=True);
backup_dataset=dataset.copy()
print("Backing up done")

In [0]:
#@title ###One-Hot Encoding
if ipLocationMapping or portTypeMapping or protocolMapping or stateMapping or serviceMapping or attackCategoryMapping is 'nominal':
  OHE = True;

if OHE:
  print("One-Hot Encoding\n\n")
  print("Number of features before OHE: {}".format(dataset.shape[1]));

  if ipLocationMapping == 'nominal':
    ohe_srciploc = pd.get_dummies(dataset.srcip,prefix='srcip')
    ohe_dstiploc = pd.get_dummies(dataset.dstip,prefix='dstip')
    dataset = pd.concat([dataset, ohe_srciploc], axis=1)
    dataset = pd.concat([dataset, ohe_dstiploc], axis=1)
    dataset = dataset.drop(columns=['srcip','dstip'])
  if protocolMapping == 'nominal':
    ohe_protocol = pd.get_dummies(dataset.proto,prefix='proto')
    dataset = pd.concat([dataset, ohe_protocol], axis=1)
    dataset = dataset.drop(columns=['proto'])
  if portTypeMapping == 'nominal':
    ohe_sporttype = pd.get_dummies(dataset.sporttype,prefix='sporttype')
    ohe_dsporttype = pd.get_dummies(dataset.dsporttype,prefix='dsporttype')
    dataset = pd.concat([dataset, ohe_sporttype], axis=1)
    dataset = pd.concat([dataset, ohe_dsporttype], axis=1)
    dataset = dataset.drop(columns=['sporttype','dsporttype'])
  if stateMapping == 'nominal':
    ohe_state = pd.get_dummies(dataset.state,prefix='state')
    dataset = pd.concat([dataset, ohe_state], axis=1)
    dataset = dataset.drop(columns=['state'])
  if serviceMapping == 'nominal':
    ohe_service = pd.get_dummies(dataset.service,prefix='service')
    dataset = pd.concat([dataset, ohe_service], axis=1)
    dataset = dataset.drop(columns=['service'])
  if attackCategoryMapping == 'nominal':
    ohe_attack_cat = pd.get_dummies(dataset.attack_cat,prefix='attack_cat')
    dataset = pd.concat([dataset, ohe_attack_cat], axis=1)
    dataset = dataset.drop(columns=['attack_cat'])

  print("Number of features after OHE: {}".format(dataset.shape[1]));

In [0]:
#@title ###Missing values correcting
ratio= 1.5 
iteration = 3 
cross_val = 2 

nan_cols = dataset.columns[dataset.isna().any()].tolist()
print("NaN columns {}\n".format(nan_cols))

nans = lambda dataset: dataset[dataset.isnull().any(axis=1)]
not_nans = lambda dataset: dataset[dataset.notnull().any(axis=1)]

def splitDataframe(data,ratio):
  index = int(len(data)//ratio)
  return data[:index]

if len(nan_cols) >0:
  nan_dict = {}

  for n in nan_cols:
    nans = dataset[dataset[n].isnull()]
    not_nans = dataset[dataset[n].notnull()]
    nan_dict[n]=not_nans.shape[0]

  sorted_nan_dict = sorted(nan_dict.items(), key=lambda kv: kv[1], reverse=True)
    
  for d in sorted_nan_dict:
    nan_rows = dataset[dataset[d[0]].isnull()]
    not_nan_rows = dataset[dataset[d[0]].notnull()]

    X_test = nan_rows.drop(nan_cols,axis=1)
    X_train = not_nan_rows.drop(nan_cols,axis=1)
    y_train = not_nan_rows[d[0]]

    X_train = splitDataframe(X_train,ratio)
    y_train = splitDataframe(y_train,ratio)

    hyperparameters = TreeBasedModelHyperparameterSelector(X_train,y_train,iteration=iteration,cross_valid=cross_val,verbose_level=2)
    print("Random Forest Regressor hyperparameters for {} are:\n{}".format(d[0],hyperparameters))
    rfr = RandomForestRegressor(**hyperparameters, verbose=0)
    rfr.fit(X_train,y_train)
    prediction = rfr.predict(X_test)
    prediction=prediction.astype(int)
    prediction=prediction.tolist()
    dataset[d[0]] = dataset[d[0]].apply(lambda x: setPredictedValue(prediction) if math.isnan(x) else x)
    nan_cols = dataset.columns[dataset.isna().any()].tolist()

print("\nNaN columns were corrected")

In [0]:
#@title ###Normalization
print("Normalization of dataset\n")
  
dataset = dataset.drop(columns=list(dataset.filter(regex='attack_cat').columns))

label_col = dataset.label;
dataset_without_labels = dataset.drop(columns=['label'])
#unchanged_dataset = dataset[dataset.columns[:8]]

#dataset_norm = dataset_without_labels[dataset_without_labels.columns[8:]]
dataset_norm = dataset_without_labels
names = dataset_norm.columns

scaler = preprocessing.StandardScaler()
scaled_dataset = scaler.fit_transform(dataset_norm)
scaled_dataset = pd.DataFrame(scaled_dataset, columns=names)
dataset_norm.update(scaled_dataset)

#norm_dataset = pd.concat([unchanged_dataset, dataset_norm.reindex(unchanged_dataset.index)], axis=1)
dataset_norm['label'] = label_col

dataset = dataset_norm
print("Done")

In [0]:
#@title ###Resampling
if resampling:
  print("Resampling dataset\n")

  X = dataset.drop('label', axis=1)
  y = dataset.label

  print("Original dataset categories shape {}".format(sorted(Counter(y).items())))
  print("Original dataset size: {}\n".format(dataset.shape[0]))

  pipe = make_pipeline(SMOTE(sampling_strategy='auto',random_state=0,n_jobs=5), ADASYN(sampling_strategy='auto',random_state=0,n_jobs=5))
  X_resampled, Y_resampled = pipe.fit_resample(X,y)

  X_resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
  resampled_dataset = X_resampled_df.assign(label=pd.Series(Y_resampled))
  dataset = resampled_dataset

  print("\nResampled dataset categories shape {}".format(sorted(Counter(Y_resampled).items())))
  print("Resampled dataset size: {}".format(dataset.shape[0]))

In [0]:
#@title ###Correlation matrix
corrmat = dataset.corr()
top_corr_features = corrmat.index
pl.figure(figsize=(60,60))
g=sns.heatmap(dataset[top_corr_features].corr(method='pearson', min_periods=1),annot=True,cmap="RdYlGn")

In [0]:
#@title ###Final dataset
attack_traffic=dataset[dataset.label.apply(lambda x: x==1)]
normal_traffic=dataset[dataset.label.apply(lambda x: x==0)]

total_features=dataset.shape[1];
total_rows=dataset.shape[0];
normal_traffic_rows=normal_traffic.shape[0];
normal_traffic_per=(100*normal_traffic_rows)/total_rows;
attack_traffic_rows=attack_traffic.shape[0];
attack_traffic_per=(100*attack_traffic_rows)/total_rows;

print("Total rows: {}\nTotal features: {}\nNormal traffic: {} ({} %)\nAttack traffic: {} ({} %)".format(total_rows, total_features, normal_traffic_rows, round(normal_traffic_per,2), attack_traffic_rows, round(attack_traffic_per,2)));

In [0]:
#@title ###Save document
saveDataset(dataset,datasetToSave)
saveDocument(notebookName)
saveDatasetProfileReport(dataset,notebookName)